In [60]:
import pandas as pd
import numpy as np
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import xgboost as xgb

In [ ]:
train_data=pd.read_csv('datasets/train.csv')
print(train_data.info())
data=np.array(train_data)

In [ ]:
categorical_data=train_data.select_dtypes(include=['object'])
numerical_data=train_data.select_dtypes(exclude=['object'])
print(categorical_data.info())
print(numerical_data.info())

In [ ]:
cat_col=[]
num_col=[]
for i in train_data.columns:
    if train_data[i].dtype=='object':
        cat_col.append(i)
    else:
        num_col.append(i)

print('categorical col: '+str(cat_col))
print('numerical col: '+str(num_col))


In [ ]:
print(train_data.isnull().sum())
print('*'*100)

In [39]:
train_data["Policy Start Date"]=pd.to_datetime(train_data["Policy Start Date"])

train_data["year"]=train_data["Policy Start Date"].dt.year
train_data["month"]=train_data["Policy Start Date"].dt.month
train_data["day"]=train_data["Policy Start Date"].dt.day

train_data.drop("Policy Start Date",axis=1,inplace=True)

In [ ]:
num_col.append("year")
num_col.append("month")
num_col.append("day")
cat_col.remove("Policy Start Date")

In [ ]:
print(train_data.info())

In [ ]:
print(train_data.isnull().sum())

In [ ]:
num_inputer=SimpleImputer(strategy='mean')
cat_inputer=SimpleImputer(strategy='most_frequent')

train_data[num_col]=num_inputer.fit_transform(train_data[num_col])
train_data[cat_col]=cat_inputer.fit_transform(train_data[cat_col])

print(train_data.isnull().sum())


In [ ]:
print(train_data.info())

In [ ]:
# encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
# for i in train_data.columns:
#   # print(i)
#   encoded_value=encoder.fit_transform(train_data[[i]])
#   encoded_value=pd.DataFrame(encoded_value,columns=encoder.get_feature_names_out([i]))
#   train_data.drop(i,axis=1,inplace=True)
#   train_data=pd.concat([train_data,encoded_value],axis=1)

In [61]:
num=train_data.columns
scaler=StandardScaler()
train_data[num]=scaler.fit_transform(train_data[num])

In [ ]:
correration_matrix=train_data.corr()
print(correration_matrix)

In [ ]:
print(correration_matrix["Premium Amount"])
train_data.drop("id",axis=1,inplace=True)

In [ ]:
y=np.log1p(train_data["Premium Amount"])
X=train_data.drop("Premium Amount",axis=1)

Now implement a regressor

In [ ]:
best_params = {
    'colsample_bytree': 0.9,
    'learning_rate': 0.01,
    'max_depth': 7,
    'n_estimators': 500,
    'subsample': 0.8
}

model=xgb.XGBRegressor(**best_params)

In [ ]:
model.fit(X,y)
predictions=model.predict(X)

accuracy=model.score(X,y)

prediction=np.expm1(predictions)